import packages

In [4]:
import os
import shutil
import glob
import json
import time

ENCODING_TEST_EIDS_PATH = "../../data/encoding_test_eids.txt"
POSE_TEST_EIDS_PATH = "../../data/pose_test_eids.txt"
ACTION_TEST_EIDS_PATH = "../../data/action_test_eids.txt"
PRETRAIN_EIDS_PATH = "../../data/pretrain_eids.txt"

DATA_DIR = "../../data/ssl/ibl-mouse-separate"

folders = os.listdir(DATA_DIR)

# read info.json file in data_dir
json_path = os.path.join(DATA_DIR, 'info.json')
with open(json_path, 'r') as f:
    info = json.load(f)
print(info)

{'dataset': 'ibl-mouse-separate', 'description': 'A self-supervised dataset for multi-view learning', 'available_views': ['rightCamera', 'leftCamera'], 'anchor_view': 'leftCamera', 'video_ids': ['9b528ad0-4599-4a55-9148-96cc1d93fb24', '15b69921-d471-4ded-8814-2adad954bcd8', '0802ced5-33a3-405e-8336-b65ebc5cb07c', 'ff96bfe1-d925-4553-94b5-bf8297adf259', 'ee40aece-cffd-4edb-a4b6-155f158c666a', 'a405053a-eb13-4aa4-850c-5a337e5dc7fd', '6c6b0d06-6039-4525-a74b-58cfaa1d3a60', '2bdf206a-820f-402f-920a-9e86cd5388a4', '8928f98a-b411-497e-aa4b-aa752434686d', '3e6a97d3-3991-49e2-b346-6948cb4580fb', 'f3ce3197-d534-4618-bf81-b687555d1883', '239dd3c9-35f3-4462-95ee-91b822a22e6b', 'd57df551-6dcb-4242-9c72-b806cff5613a', 'e2b845a1-e313-4a08-bc61-a5f662ed295e', '687017d4-c9fc-458f-a7d5-0979fe1a7470', '6899a67d-2e53-4215-a52a-c7021b5da5d4', 'c51f34d8-42f6-4c9c-bb5b-669fd9c42cd9', '746d1902-fa59-4cab-b0aa-013be36060d5', 'e45481fa-be22-4365-972c-e7404ed8ab5a', 'f312aaec-3b6f-44b3-86b4-3a0c119c0438', '03d9

Remove test folders data from ssl pretraining dataset

In [5]:
# list folders in DATA_DIR

for test_eids_path in [ENCODING_TEST_EIDS_PATH, POSE_TEST_EIDS_PATH, ACTION_TEST_EIDS_PATH]:
    with open(test_eids_path, "r") as f:
        eids = f.readlines()
    print(f'start processing {test_eids_path}')
    eid_list = []
    for eid in eids:
        eid = eid.strip()
        eid_list.append(eid)
    print(f'number of eids to remove: {len(eid_list)}')
    print(eid_list)

    for folder in folders:
        if folder in eid_list:
            print(f'removing {folder}')
            # remove this folder and all its contents
            try:
                shutil.rmtree(os.path.join(DATA_DIR, folder))
            except Exception as e:
                print(f'Error removing {folder}: {e}')

# list files in each folder

start processing ../../data/encoding_test_eids.txt
number of eids to remove: 10
['d23a44ef-1402-4ed7-97f5-47e9a7a504d9', 'db4df448-e449-4a6f-a0e7-288711e7a75a', '3638d102-e8b6-4230-8742-e548cd87a949', '4b7fbad4-f6de-43b4-9b15-c7c7ef44db4b', '03d9a098-07bf-4765-88b7-85f8d8f620cc', '0841d188-8ef2-4f20-9828-76a94d5343a4', '9b528ad0-4599-4a55-9148-96cc1d93fb24', 'f140a2ec-fd49-4814-994a-fe3476f14e66', '687017d4-c9fc-458f-a7d5-0979fe1a7470', 'd04feec7-d0b7-4f35-af89-0232dd975bf0']
start processing ../../data/pose_test_eids.txt
number of eids to remove: 10
['6e18b50d-8ead-44a8-81c7-e09d2e2df3c0', '7af49c00-63dd-4fed-b2e0-1b3bd945b20b', '26aa51ff-968c-42e4-85c8-8ff47d19254d', '30e5937e-e86a-47e6-93ae-d2ae3877ff8e', '56b57c38-2699-4091-90a8-aba35103155e', '862ade13-53cd-4221-a3fa-dda8643641f2', '73918ae1-e4fd-4c18-b132-00cb555b1ad2', 'a52f5a1b-7f45-4f2c-89a9-fb199d2a0d63', 'a92c4b1d-46bd-457e-a1f4-414265f0e2d4', 'ebe090af-5922-4fcd-8fc6-17b8ba7bad6d']
start processing ../../data/action_test_ei

Check how many folders in the DATA_DIR, and return the number of total png files

In [7]:
# list all folders in DATA_DIR
# must only folders not contain json or files
folders = os.listdir(DATA_DIR)
num_folders = 0
pretrain_eids = []
for folder in folders:
    if os.path.isdir(os.path.join(DATA_DIR, folder)):
        num_folders += 1
        pretrain_eids.append(folder)
# save pretrain_eids to a text file
with open(PRETRAIN_EIDS_PATH, "w") as f:
    for eid in pretrain_eids:
        f.write(f"{eid}\n")

print(f"Number of folders: {num_folders}")

# recursively count total number of png files in each folder
# walk through each folder and count the number of png files
# globally get all the png files

png_files = glob.glob(os.path.join(DATA_DIR, "**/*.png"), recursive=True)
print(f"Number of png files: {len(png_files)}")

info['number_of_videos'] = num_folders
info['total_ssl_frames'] = len(png_files)
info['timestamp'] = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
info['video_ids'] = pretrain_eids

# save info.json file in data_dir
with open(json_path, 'w') as f:
    json.dump(info, f, indent=4)

Number of folders: 67


Number of png files: 107192
